In [1]:
import cv2
import numpy as np


In [5]:
cap = cv2.VideoCapture(0)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
img = cv2.imread("maxresdefault.jpg", 1)
cv2.imshow('hardbass', img)
while True:
    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()
        break
    if key == 32:
        cv2.putText(img, "hardbass for life", (10, 700), font, 2, (255, 255, 255), 5, cv2.LINE_AA)
        cv2.destroyAllWindows()
        cv2.imshow('hardbass for life', img)

In [ ]:
while True:
    count += 1
    ret, frame = cap.read()
    cv2.putText(frame, str(count%10), (20, 180), font, 4, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
while True:
    ret, frame = cap.read()
    a = np.array(frame)
    b = np.average(a)
    print (.shape)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
